In [1]:
import pandas as pd
import requests
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

In [3]:
r = requests.get(url)
json = r.json()

In [4]:
elements_df = pd.DataFrame(json['elements'])
elements_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,100.0,100.0,232223,0,0,0,0,0,4,0.5,...,428,68,505,48,505,48,204,38,0.0,0.0
1,NaN,NaN,58822,0,0,0,0,0,2,0.5,...,567,155,333,111,333,111,258,96,0.0,0.0
2,0.0,0.0,153256,-1,1,-1,1,0,3,0.0,...,543,286,440,155,440,155,424,136,0.0,0.0
3,NaN,NaN,438098,-1,1,-1,1,0,3,0.5,...,148,94,648,277,648,277,418,133,0.0,0.0
4,NaN,NaN,226597,-1,1,-1,1,0,2,1.5,...,300,33,241,84,274,103,15,6,0.0,0.0


In [5]:
slim_elements_df = elements_df[['web_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]
slim_elements_df.head()

,web_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points
0,Balogun,1,4,0.7,45,0,7126,0.0,0
1,Cédric,1,2,0.4,40,0,3100,0.0,0
2,M.Elneny,1,3,0.1,44,0,2869,0.0,0
3,Fábio Vieira,1,3,0.1,54,0,608,0.0,0
4,Gabriel,1,2,24.0,49,24,90123,0.4,2


In [6]:
def playerSubbed(minutes):
    max_possible = slim_elements_df.minutes.max()
    
    if max_possible - minutes > 0:
        sub_status = 'subbed'
    else:
        sub_status = 'not_subbed'
    return sub_status

In [7]:
slim_elements_df['sub_status'] = slim_elements_df.minutes.apply(playerSubbed)
slim_elements_df.sample(10)

,web_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,sub_status
623,Cornet,19,3,0.1,54,1,218,0.2,1,subbed
420,Kovačić,13,3,1.1,50,157,18253,1.2,6,subbed
527,Horvath,16,1,0.1,45,0,668,0.0,0,subbed
260,Johnstone,8,1,2.1,45,180,21329,1.8,8,not_subbed
15,Pepe,1,3,0.1,49,0,430,0.0,0,subbed
283,Garner,9,3,0.0,50,172,408,0.8,4,subbed
330,Raúl,10,4,1.0,54,142,10925,0.6,3,subbed
302,Danjuma,9,3,0.2,55,70,2362,0.4,2,subbed
286,Gray,9,3,0.3,54,0,1092,0.0,0,subbed
248,Humphreys,7,2,0.0,40,0,110,0.0,0,subbed


In [8]:
slim_elements_df.loc[slim_elements_df.sub_status == 'not_subbed']
slim_elements_df.sample(10)

,web_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,sub_status
173,Brownhill,6,3,0.2,49,1,626,0.2,1,subbed
415,Doyle,13,3,0.0,45,0,168,0.0,0,subbed
341,Darwin,11,4,8.6,74,28,78130,0.3,2,subbed
118,Jensen,4,3,0.7,54,138,4912,1.3,7,subbed
522,Dräger,16,2,0.1,40,0,662,0.0,0,subbed
4,Gabriel,1,2,24.0,49,24,90123,0.4,2,subbed
183,McNally,6,2,0.0,40,0,0,0.0,0,subbed
250,Andersen,8,2,2.9,45,180,140674,2.4,11,not_subbed
434,Gvardiol,13,2,4.6,50,101,154059,1.4,7,subbed
608,Son,18,3,5.9,90,164,123914,0.6,5,subbed


In [9]:
slim_elements_df.selected_by_percent = slim_elements_df.selected_by_percent.astype(float)

In [10]:
def findValue (selected_by_percent, total_points):
    selected_by_limit = 10
    top_90_percentile = slim_elements_df.total_points.quantile(0.9)
    
    if (selected_by_percent <= selected_by_limit) & (total_points >= top_90_percentile) :
        scout_flag = 'interesting'
    else :
        scout_flag = 'meh'
    return scout_flag

In [11]:
slim_elements_df['scout_flag'] = slim_elements_df.apply(lambda x: findValue(x['selected_by_percent'], x['total_points']),axis=1)
slim_elements_df.sample(10)

,web_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,sub_status,scout_flag
342,Diogo J.,11,3,3.4,80,140,84306,1.6,13,subbed,interesting
501,Watts,15,2,0.1,40,0,0,0.0,0,subbed,meh
274,Matheus França,8,3,0.0,50,0,63,0.0,0,subbed,meh
352,Mac Allister,11,3,7.1,60,147,49885,-0.2,-1,subbed,meh
649,Chiquinho,20,3,0.0,45,0,0,0.0,0,subbed,meh
125,Roerslev,4,2,0.1,45,30,147,0.4,2,subbed,meh
529,Kouyaté,16,3,0.0,50,2,94,0.4,2,subbed,meh
324,Robinson,10,2,0.7,45,180,12059,1.6,7,not_subbed,interesting
602,Rodon,18,2,0.1,40,0,0,0.0,0,subbed,meh
403,Francis-Clarke,12,2,0.0,40,0,400,0.0,0,subbed,meh


In [12]:
slim_elements_df.loc[(slim_elements_df.scout_flag == 'interesting')&(slim_elements_df.sub_status == 'not_subbed')]
slim_elements_df.sample(10)

,web_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,sub_status,scout_flag
615,Van de Ven,18,2,0.6,45,180,31554,1.8,8,not_subbed,interesting
239,T.Silva,7,2,2.9,50,180,55049,0.6,3,not_subbed,meh
425,Ortega Moreno,13,1,0.6,40,0,2223,0.0,0,subbed,meh
449,Heaton,14,1,0.1,40,0,158,0.0,0,subbed,meh
197,Berge,6,3,0.2,50,90,1417,0.4,2,subbed,meh
147,Karbownik,5,2,0.0,40,0,0,0.0,0,subbed,meh
294,Onana,9,3,0.2,50,180,1828,0.8,4,not_subbed,meh
597,Ndombele,18,3,0.5,44,0,4997,0.0,0,subbed,meh
458,Mengi,14,2,0.1,40,0,332,0.0,0,subbed,meh
243,Disasi,7,2,5.6,50,180,410207,2.2,11,not_subbed,interesting
